Generate a COCO keypoints dataset of black images with circles on it for integration testing of the keypoint detector. 


In [192]:
import cv2
import numpy as np 
from airo_dataset_tools.data_parsers.coco import CocoKeypointAnnotation, CocoImage, CocoKeypointCategory, CocoKeypointsDataset
import pathlib
!pip install distinctipy
import distinctipy

In [193]:
n_images = 500
n_categories = 2
max_category_instances_per_image = 2

image_resolution = (128, 128)
circle_radius = 1

In [194]:


DATA_DIR = pathlib.Path("./dummy_dataset")
DATA_DIR.mkdir(exist_ok=True)
IMAGE_DIR = DATA_DIR / "images"
IMAGE_DIR.mkdir(exist_ok=True)

In [195]:
categories = []
for category_idx in range(n_categories):
    coco_category = CocoKeypointCategory(
        id=category_idx,
        name=f"dummy{category_idx}",
        supercategory=f"dummy{category_idx}",
        keypoints=[f"dummy{category_idx}"]
    )
    categories.append(coco_category)

In [196]:
category_colors = distinctipy.get_colors(n_categories)
category_colors = [tuple([int(c * 255) for c in color]) for color in category_colors]

In [197]:
coco_images = []
cococ_annotations = []

coco_instances_coutner = 0
for image_idx in range(n_images):
    img = np.zeros((image_resolution[1],image_resolution[0],3), dtype=np.uint8)
    coco_images.append(CocoImage(id=image_idx, file_name=f"images/img_{image_idx}.png", height=image_resolution[1], width=image_resolution[0]))
    for category_idx in range(n_categories):
        n_instances = np.random.randint(0, max_category_instances_per_image+1)
        for instance_idx in range(n_instances):
            x = np.random.randint(2, image_resolution[0])
            y = np.random.randint(2, image_resolution[1])
            img = cv2.circle(img, (x, y), circle_radius, category_colors[category_idx], -1)
            cococ_annotations.append(CocoKeypointAnnotation(
                id=coco_instances_coutner,
                image_id=image_idx,
                category_id=category_idx,
                # as in coco datasets: zero-index, INT keypoints.
                # but add some random noise (simulating dataset with the exact pixel location instead of the zero-index int location)
                # to test if the detector can deal with this
                keypoints=[x + np.random.rand(1).item(), y + np.random.rand(1).item(), 1],
                num_keypoints=1,
            ))
            coco_instances_coutner += 1

    cv2.imwrite(str(DATA_DIR / "images"/f"img_{image_idx}.png"), img)

coco_dataset = CocoKeypointsDataset(
    images=coco_images,
    annotations=cococ_annotations,
    categories=categories,
)

with open(DATA_DIR / "dummy_dataset.json", "w") as f:
    f.write(coco_dataset.json())
